In [6]:
!git clone https://github.com/nishantkushwaha-1999/AlphaZero_C4.git

Cloning into 'AlphaZero_C4'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 33 (delta 7), reused 25 (delta 4), pack-reused 0
Receiving objects: 100% (33/33), 39.27 KiB | 3.57 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [2]:
!pip install torchinfo

In [3]:
import os
path = os.getcwd() + "/AlphaZero_C4"
os.chdir(path)

In [1]:
import torch
from Game.C4 import Game_C4
from AlphaZero.model import ResNet_C4
from AlphaZero.AlphaZero import AlphaZero
from torchinfo import summary

pygame 2.5.2 (SDL 2.28.3, Python 3.10.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


# Train AlphaZero_C4

### Train from Scratch

In [4]:
game_c4 = Game_C4()

args = {
    'board_dim': (game_c4.rows, game_c4.columns),
    'n_actions': game_c4.action_size,
    'n_res_blocks': 1,
    'n_hidden': 16,
    'C': 2,
    'lr': 0.001,
    'weight_decay': 0.001,
    'num_searches': 100,
    'n_iters': 2,
    'n_selfPlay': 20,
    'n_parallel': True,
    'n_parallel_games': 100,
    'epochs': 2,
    'batch_size': 2,
    'temperature': 1.25,
    'dirichlet_epsilon': 0.25,
    'dirichlet_alpha': 0.03
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNet_C4(
    board_dim=args['board_dim'],
    n_actions=args['n_actions'],
    n_res=args['n_res_blocks'],
    n_hidden=args['n_hidden'],
    device=device
)
optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'], weight_decay=args['weight_decay'])
architecture = summary(model, input_size=(args['batch_size'], 3, game_c4.rows, game_c4.columns), verbose=0)
print(architecture)

Layer (type:depth-idx)                   Output Shape              Param #
ResNet_C4                                [2, 1]                    --
├─Sequential: 1-1                        [2, 16, 6, 7]             --
│    └─Conv2d: 2-1                       [2, 16, 6, 7]             448
│    └─BatchNorm2d: 2-2                  [2, 16, 6, 7]             32
│    └─ReLU: 2-3                         [2, 16, 6, 7]             --
├─ModuleList: 1-2                        --                        --
│    └─ResBlock: 2-4                     [2, 16, 6, 7]             --
│    │    └─Conv2d: 3-1                  [2, 16, 6, 7]             2,320
│    │    └─BatchNorm2d: 3-2             [2, 16, 6, 7]             32
│    │    └─Conv2d: 3-3                  [2, 16, 6, 7]             2,320
│    │    └─BatchNorm2d: 3-4             [2, 16, 6, 7]             32
├─Sequential: 1-3                        [2, 1]                    --
│    └─Conv2d: 2-5                       [2, 32, 6, 7]             4,640
│    

In [5]:
alphazero = AlphaZero()
alphazero.learn(model, optimizer, game_c4, args, n_parallel=args['n_parallel'])

Iter 1 of 2
Self Play:


0it [00:00, ?it/s]


Model Train:


0it [00:00, ?it/s]
0it [00:00, ?it/s]

Model saved at: /Volumes/Storage/Git Repos/AlphaZero_C4/saved_models/****_1_Connect_4.pt
Iter 2 of 2
Self Play:


0it [00:00, ?it/s]


Model Train:


0it [00:00, ?it/s]
0it [00:00, ?it/s]


Model saved at: /Volumes/Storage/Git Repos/AlphaZero_C4/saved_models/****_2_Connect_4.pt


### Load & Train

In [2]:
game_c4 = Game_C4()

alphazero = AlphaZero()
alphazero.load_and_resume(game_c4, "/Volumes/Storage/Git Repos/AlphaZero_C4/saved_models/")

Iter 3 of 10
Self Play:


0it [00:00, ?it/s]


Model Train:


0it [00:00, ?it/s]
0it [00:00, ?it/s]


Model saved at: /Volumes/Storage/Git Repos/AlphaZero_C4/saved_model/****_3_Connect_4.pt
Iter 4 of 10
Self Play:


...

Model saved at: /Volumes/Storage/Git Repos/AlphaZero_C4/saved_model/****_7_Connect_4.pt
Iter 8 of 10
Self Play:


0it [00:00, ?it/s]


Model Train:


0it [00:00, ?it/s]
0it [00:00, ?it/s]


Model saved at: /Volumes/Storage/Git Repos/AlphaZero_C4/saved_model/****_8_Connect_4.pt
